In [1]:
# Most basic stuff for EDA.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Core packages for text processing.
import string
import re

# Libraries for text preprocessing.
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KIIT\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\KIIT\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\KIIT\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
# Loading some sklearn packaces for modelling.
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import LatentDirichletAllocation, NMF # not actively using
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

# Utility
import logging
import itertools

In [4]:
# Core packages for general use throughout the notebook.
import random
import warnings
import time
import datetime

# For customizing our plots.
from matplotlib.ticker import MaxNLocator
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches

In [5]:
# for build our model
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from transformers import BertTokenizer, TFBertModel

# Setting some options for general use.
import os
stop = set(stopwords.words('english'))
plt.style.use('fivethirtyeight')
sns.set(font_scale=1.5)
pd.options.display.max_columns = 250
pd.options.display.max_rows = 250
warnings.filterwarnings('ignore')

In [6]:
df = pd.read_csv('updated_dataset.csv')
df.head()

,Review,neg,neu,pos,compound,Labels
0,"Unfortunately, the frustration of being Dr. Go...",0.124,0.852,0.024,-0.8997,1
1,Been going to Dr. Goldberg for over 10 years. ...,0.000,0.957,0.043,0.6249,4
2,I don't know what Dr. Goldberg was like before...,0.141,0.770,0.090,-0.9439,1
3,I'm writing this review to give you a heads up...,0.045,0.865,0.089,0.6678,4
4,All the food is great here. But the best thing...,0.000,0.512,0.488,0.9958,5


In [7]:
#map target label to string

In [8]:
decode_map = {1: "ANGRY", 2: "NEGATIVE", 3: "NEUTRAL", 4: "POSITIVE", 5: "GREAT"}
def decode_sentiment(Labels):
    return decode_map[int(Labels)]

In [9]:
%%time
df.Labels = df.Labels.apply(lambda x: decode_sentiment(x))

CPU times: total: 219 ms
Wall time: 221 ms


# cleaning Text

In [10]:
def remove_stopwords(text):
    tokens = []
    for token in text.split():
        if token not in stop:
            tokens.append(token)
    return " ".join(tokens)


def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)

def remove_html(text):
    html = re.compile(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    return re.sub(html, '', text)


def remove_punct(text):
    table = str.maketrans('', '', string.punctuation)
    return text.translate(table)

# Applying helper functions
df['text_clean'] = df['Review'].apply(lambda x: remove_stopwords(x))
df['text_clean'] = df['text_clean'].apply(lambda x: remove_URL(x))
df['text_clean'] = df['text_clean'].apply(lambda x: remove_html(x))
df['text_clean'] = df['text_clean'].apply(lambda x: remove_punct(x))


In [11]:
df.head()


,Review,neg,neu,pos,compound,Labels,text_clean
0,"Unfortunately, the frustration of being Dr. Go...",0.124,0.852,0.024,-0.8997,ANGRY,Unfortunately frustration Dr Goldbergs patient...
1,Been going to Dr. Goldberg for over 10 years. ...,0.000,0.957,0.043,0.6249,POSITIVE,Been going Dr Goldberg 10 years I think I one ...
2,I don't know what Dr. Goldberg was like before...,0.141,0.770,0.090,-0.9439,ANGRY,I know Dr Goldberg like moving Arizona let tel...
3,I'm writing this review to give you a heads up...,0.045,0.865,0.089,0.6678,POSITIVE,Im writing review give heads see Doctor The of...
4,All the food is great here. But the best thing...,0.000,0.512,0.488,0.9958,GREAT,All food great here But best thing wings Their...


# Setup environment to build model

In [12]:
os.environ["WANDB_API_KEY"] = "0" ## to silence warning

In [13]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    strategy = tf.distribute.get_strategy() # for CPU and single GPU
    print('Number of replicas:', strategy.num_replicas_in_sync)

Number of replicas: 1


In [14]:
# hyperparameters
max_length = 128
batch_size = 128


In [15]:
# Bert Tokenizer
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)

# splitting the data

In [16]:
train_df, test = train_test_split(df, test_size=0.01, random_state=42) 
x_train, dev = train_test_split(train_df, test_size=0.01, random_state=42)

In [17]:
train = x_train[:5000]

In [18]:
labels = train.Labels.unique().tolist()
labels

['GREAT', 'ANGRY', 'POSITIVE', 'NEUTRAL', 'NEGATIVE']

In [19]:
encoder = LabelEncoder()
encoder.fit(train.Labels.tolist())

y_train = encoder.transform(train.Labels.tolist())
y_test = encoder.transform(test.Labels.tolist())
y_dev = encoder.transform(dev.Labels.tolist())

y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)
y_dev = y_dev.reshape(-1,1)

print("y_train",y_train.shape)
print("y_test",y_test.shape)

y_train (5000, 1)
y_test (5600, 1)


In [20]:
#takes a list of strings as input and tokenizes into sequence of integer ID
def bert_encode(data):
    tokens = tokenizer.batch_encode_plus(data, max_length=max_length, padding='max_length', truncation=True) 
    
    return tf.constant(tokens['input_ids'])

In [21]:
train_encoded = bert_encode(train.text_clean)
dev_encoded = bert_encode(dev.text_clean)


train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((train_encoded, y_train))
    .shuffle(128)
    .batch(batch_size)
)

dev_dataset = (
    tf.data.Dataset
    .from_tensor_slices((dev_encoded, y_dev))
    .shuffle(128)
    .batch(batch_size)
)

# MODEL

In [22]:
def bert_model():

    bert_encoder = TFBertModel.from_pretrained(model_name)
    input_word_ids = tf.keras.Input(shape=(max_length,), dtype=tf.int32, name="input_ids")
    last_hidden_states = bert_encoder(input_word_ids)[0]   
    x = tf.keras.layers.SpatialDropout1D(0.2)(last_hidden_states)
    x = tf.keras.layers.Conv1D(64, 3, activation='relu')(x)
    x = tf.keras.layers.Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2))(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    

    
    outputs = tf.keras.layers.Dense(5, activation='softmax')(x)
    model = tf.keras.Model(input_word_ids, outputs)
   
    return model

In [23]:
with strategy.scope():
    model = bert_model()
    adam_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
    model.compile(loss='sparse_categorical_crossentropy',optimizer=adam_optimizer,metrics=['accuracy'])

    model.summary()

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_ids (InputLayer)      [(None, 128)]             0         
                                                                 
 tf_bert_model (TFBertModel)  TFBaseModelOutputWithPoo  109482240
                             lingAndCrossAttentions(l            
                             ast_hidden_state=(None,             
                             128, 768),                          
                              pooler_output=(None, 76            
                             8),                                 
                              past_key_values=None, h            
                             idden_states=None, atten            
                             tions=None, cross_attent            
                             ions=None)                          
                                                             

In [27]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

callbacks = [ ReduceLROnPlateau(monitor='val_loss', patience=5, cooldown=0),
              EarlyStopping(monitor='val_accuracy', min_delta=1e-5, patience=5)]  
#early stopping to stop the training process once the validation loss starts to increase, 
#indicating that the model is starting to overfit.

In [30]:
# Start train
#history = model.fit(
 #   train_dataset,
  #  batch_size=batch_size,
  #  epochs=6,
   # validation_data=dev_dataset,
    #verbose=1,
    #callbacks = callbacks)

In [ ]:
# SAVE MODEL WEIGHTS
#model.save_weights(f'sentiment_weights_v1.h5')

In [ ]:
model.save('sentiment_model_bert')

INFO:tensorflow:Assets written to: sentiment_model_bert\assets


INFO:tensorflow:Assets written to: sentiment_model_bert\assets


In [33]:
#from tensorflow.keras.models import load_model
#model = load_model('sentiment_model_bert')

In [34]:
# LOAD MODEL WEIGHTS
model.load_weights('sentiment_weights_v1.h5')

In [35]:
'''
def plot_graphs(sentiment_model_bert.history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()
   
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")
'''

'\ndef plot_graphs(sentiment_model_bert.history, string):\n    plt.plot(history.history[string])\n    plt.plot(history.history[\'val_\'+string])\n    plt.xlabel("Epochs")\n    plt.ylabel(string)\n    plt.legend([string, \'val_\'+string])\n    plt.show()\n   \nplot_graphs(history, "accuracy")\nplot_graphs(history, "loss")\n'

In [36]:
# LOAD MODEL WEIGHTS
model.load_weights('sentiment_weights_v1.h5')

In [37]:
def get_label(score):
    if score < 0.2:
        return "ANGRY"
    elif 0.2 <= score < 0.4:
        return "NEGATIVE"
    elif 0.4 <= score < 0.6:
        return "NEUTRAL"
    elif 0.6 <= score < 0.8 :
        return "HAPPY"
    else:
        return "GREAT"

#def decode_sentiment(scores):
    #label = np.argmax(scores)
    #return get_label(label)

In [38]:
def predict(text, include_neutral=True):
    start_at = time.time()
    # Tokenize text
    x_encoded = bert_encode([text])
    # Predict
    score = model.predict([x_encoded])[0]
    # Decode sentiment
    label_index = np.argmax(score)
    if include_neutral:
        label = get_label(label_index)
    else:
        if label_index == 2: # Neutral label index is 2
            label = None
        else:
            label = get_label(label_index)
    
    return {"label": label,
            "score": score.tolist(),
            "elapsed_time": time.time()-start_at}


In [39]:
def print_sentiment_label(score):
    sentiment_label = decode_sentiment(score)
    print(sentiment_label)

In [40]:
predict("I hate the economy")

1/1 [==============================] - 5s 5s/step


{'label': 'ANGRY',
 'score': [0.5851972699165344,
  0.11982598900794983,
  0.05445199832320213,
  0.14637352526187897,
  0.09415127336978912],
 'elapsed_time': 4.979530096054077}

In [133]:
score_array = np.array([0.58519727, 0.11982599, 0.054452  , 0.14637353, 0.09415127])
print_sentiment_label(score_array)

ANGRY


In [139]:
predict("Had a song stuck in my head. Thirty seconds later I'm listening to it, thanks to the internet,\
        and Apple/YouTube Music. In the bad old days I'd browse record stores for hours in the hope that the title might jog my memory.\
        It really is a wonderful time to be alive!")

1/1 [==============================] - 0s 392ms/step


{'label': 'GREAT',
 'score': [0.275446355342865,
  0.4498341977596283,
  0.054373595863580704,
  0.11034145206212997,
  0.11000437289476395],
 'elapsed_time': 0.4830925464630127}

In [141]:
#score_array = np.array([0.27544636, 0.4498342 , 0.0543736 , 0.11034145, 0.11000437])
#print_sentiment_label(score_array)

In [148]:
predict("For the third time in four years, the Warriors are champions once again.\
This time, they wasted no time in the NBA Finals, dispatching LeBron James and the Cavs in four straight games.\
Here’s how they sealed the championship in Game 4. https://twitter.com/i/moments/1005197277663641600", True)

1/1 [==============================] - 0s 402ms/step


{'label': 'ANGRY',
 'score': [0.4077552556991577,
  0.23904335498809814,
  0.0668078288435936,
  0.14405415952205658,
  0.1423393189907074],
 'elapsed_time': 0.5065600872039795}

In [143]:
predict("these r not ur problems dear!!! these r ur x bf's commitng suicide")

1/1 [==============================] - 0s 409ms/step


{'label': 'ANGRY',
 'score': [0.6300213932991028,
  0.10498473048210144,
  0.04956117644906044,
  0.13546928763389587,
  0.07996342331171036],
 'elapsed_time': 0.5037593841552734}

In [146]:
predict("I found some old Reddit post in which one guy from english-speaking country complains that\
the names in The Witcher books are 'too difficult' and non- intuitive for english speaker.\
Man, let me introduce you to 'The books werent written only/for english speakers.'' #witcher", True)

1/1 [==============================] - 0s 427ms/step


{'label': 'ANGRY',
 'score': [0.5757659077644348,
  0.14012154936790466,
  0.0511014498770237,
  0.14511822164058685,
  0.08789292722940445],
 'elapsed_time': 0.5224635601043701}

In [145]:
predict("happiest alive in the whole world, happiest alive in the whole world, happiest alive in the whole world")

1/1 [==============================] - 0s 416ms/step


{'label': 'GREAT',
 'score': [0.22620899975299835,
  0.4470811188220978,
  0.06442898511886597,
  0.1052616611123085,
  0.15701927244663239],
 'elapsed_time': 0.5034942626953125}

# Test

In [41]:
test_encoded = bert_encode(test.text_clean)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(test_encoded)
    .batch(batch_size)
)

y_pred = []
predicted_texts = model.predict(test_dataset, batch_size=batch_size)
predicted_texts_binary = tf.cast(tf.round(predicted_texts), tf.int32).numpy().flatten()

44/44 [==============================] - 1949s 44s/step


In [42]:
%%time
scores = model.evaluate(test_encoded, y_test, batch_size=batch_size)
print()
print("ACCURACY:",scores[1])
print("LOSS:",scores[0])

44/44 [==============================] - 2683s 61s/step - loss: 0.8598 - accuracy: 0.7380

ACCURACY: 0.7380357384681702
LOSS: 0.8598493337631226
CPU times: total: 3h 52min 40s
Wall time: 44min 43s


In [44]:
y_pred = []
for i in range(predicted_texts.shape[0]):
    y_pred.append(np.argmax(predicted_texts[i]))

In [45]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.67      0.82      0.74      1554
           1       0.77      0.94      0.85      3038
           2       0.00      0.00      0.00       187
           3       0.00      0.00      0.00       300
           4       0.32      0.01      0.03       521

    accuracy                           0.74      5600
   macro avg       0.35      0.35      0.32      5600
weighted avg       0.64      0.74      0.67      5600



In [54]:
print('Precision: %.4f' % precision_score(y_test, y_pred, average='weighted'))
print('Recall: %.4f' % recall_score(y_test, y_pred, average='weighted'))
print('Accuracy: %.4f' % accuracy_score(y_test, y_pred))
print('F1 Score: %.4f' % f1_score(y_test, y_pred, average='weighted'))

Precision: 0.6361
Recall: 0.7380
Accuracy: 0.7380
F1 Score: 0.6677


# Error Analysis

In [60]:
decode_map = {0: "ANGRY", 1: "NEGATIVE", 2: "NEUTRAL", 3: "POSITIVE", 4: "GREAT"}
def decode_sentiment(label):
    return decode_map[int(label)]

In [61]:
df = pd.DataFrame(test.Review, columns=["Review"])
df["actual"] = test.Labels
df["predicted"] = y_pred
df.predicted = df.predicted.apply(lambda x: decode_sentiment(x))
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)
#incorrect = df[df["actual"] != df["predicted"]]
#incorrect[10:20]

In [62]:
incorrect = df[df["actual"] != df["predicted"]]
incorrect[10:20]

,Review,actual,predicted
505575,"I love all my carbs toasted. I would rather be late for work and take the time to toast my bagel than eat it untoasted. This place drew me in because all their subs are TOASTED. Toasted to perfection, served warm, with just a little bit of golden brown starting to appear on the bread. Mmmm....heaven. \n\nThis place has a fun vibe. Reggae music playing, really chill workers, plenty of seating inside. Their sandwiches come in 3 sizes 4 inches up to 12 inches. Nicely priced and a pretty extensive menu. \n\nI got the Kali Mist which was delicious. Had a bit of kick to it, nicely sauced. I hate dry sandwiches. Can't wait to come back and try all their subs. They have a fun selection of desserts rice krispies but also captain crunch krispy bars, cinnamon toast crunch bars, and fruity pebble bars. \n\nSorry Jimmy John's, Which Wich, Firehouse Subs, Quizno's... every other sub place on state street I'm starting a love affair with Cheeba Hut.",GREAT,NEGATIVE
191966,"Was here about 5:30 pm to 8:30 pm on Mon/16 Dec with a few buddies for their Monday Night Football specials:\n- $1 drafts and they have a big draft beer selection (Blue Moon, etc)\n- 1/2 off their sushi rolls/etc, foods pretty good\n- Friendly staff/bartender\n- Nice decor with a fish tank in the center of the bar\nI'll be back for sure.",GREAT,NEGATIVE
5799,"I dig the melting pot, i mean, its fun. Fondue=fun. This was another \""Yay my birthday is during restaurant week!\"" dinner for me. The cheeses, meat course and salad are good but lets be honest that dessert is where its at. Id get all 3 courses as dessert if i could. A big pot of chocolate and then cheesecake, rise crispies, bananas, strawberries, red velvet cake and marshmallows all to dip into it. Heavenly is an understatement. \nI also dig that your in control of your own cooking times. So what if i cook my steak for 10 seconds...rare is good yo...\nGoing with a large group is smart. Then you can really sample a lot of the stuff on the menu. Our party had 8 peeps in it so we got 3 different cheeses, cooking styles and chocolates. Service was aiiiiiight considering it was a large party, but it coulda been better. I normally wouldn't be able to afford more than one course here but at $30 for everything during restaurant week it's a hella great deal.",GREAT,NEGATIVE
251295,"After a bad experience at our Favorite Restaurant Roca Akor in Scottsdale, we turned to Yelp and Roka's very capable manager Charles responded within an hour, invited us back and more than made it right. We knew they had great food and now, thanks to Charles, we know they have great management. We highly recommend it.",GREAT,NEGATIVE
502076,"I came here on a Saturday night in the summer around 930. They quoted us a wait time of 70 minutes. We had our hearts set on this place, so we said we'd wait. But the best part was that they take your phone number and text you when your table is ready. This was awesome! We got to hang out in the casino and get drinks until our table was ready. Once we were seated we were shown the drink menu. The drinks all looked amazing. Unfortunately, I was the DD for the night. We ordered parmesan (sp?) truffle fries and they were AMAZING! No joke, the best fries I've ever had in my life. The burgers were absolutely delicious. They were cooked to perfection. I hardly ever order a burger medium, and get a burger medium. It's always too well done. But this was perfect. Two friends I was there with ordered their's medium rare and you could see that theirs were absolutely medium rare and noticeably different from my medium. My husband and I keep a list of our top 5 burger places (similar to 'the list' from Friends), and this place shot to the top. We also ordered the Oreo shake and the sticky toffee pudding for dessert. Sticky toffee pudding is one of my all time favorites, but it was not at all good here. The shake was good, but I wish they had a malt option. Still, the sub-par desserts weren'

In [66]:
correct = df[df['actual'] == df['predicted']]
correct.head(30)

,Review,actual,predicted
223092,The wait time for an appointment is ridiculous. Been waiting over an hour and a half for my scheduled appointment time. These people do not value patients time,ANGRY,ANGRY
110270,I did not like this hotel at all. It's very old and not comforts in it. \nThe good thing is that it was cheap but at the time was like a room just to sleep! \nThere is no view at all and while you are in the Vegas it should not be those kind of rooms. \nWhen we came in to the room we just sow a trash cans and an a conditioner staff...!!!\nBut the Casino and staff were good and cute ...it's was almost like we were in the animation movies...but inside the casino...,NEGATIVE,NEGATIVE
99132,"Have to agree; probably the worst place for \""seafood\"" around. And the corporate atmosphere reminds me of outback or chilis.... enough said.",ANGRY,ANGRY
136813,"Very expensive!!!\nSound and environment was OK.\n$22 for two tickets is just not worth it, I would rather watch a movie at home where it is also more comfortable.\nWe went on a Saturday night and most of the theater was empty, only 2 other people were in the theater/movie we watched, the parking lot was also less than half full while in the past it used to be full and hard to find a parking spot let alone by the door.\nThey need to drop the price by at least 50% before I would go back...",ANGRY,ANGRY
239618,"Too much bad attitude, unprofessional, and mediocre results. Not to mention that most of the \""nail technicians\"" there cannot properly re-create the nail art that they up-sell. I had to remove \""art\"" and repaint my toenails. \n\nIt's ALMOST worth the $20. If it cost any more, I would never go there again.",ANGRY,ANGRY
401346,"So this weekend my 2 cousins and I decided we wanted to get our hair and makeup done at Platinum Entourage. We made reservations for 6pm for three people, assuming that we were going to get our hair done @ 6pm and then proceed to makeup done. Well we come to find that there is only one makeup artist and we all get our hair and makeup done at different times. Most companies I have gone to in Hollywood they normally will have enough staff members to fulfill the clients' need, and not try to \""flip-flop\"" (that's what prick called it) everyone around because they only have one makeup artist. According to the prick, they just \""flip-flop\"" everyone around regardless of the time you made your appointment, so if you made it at 6pm chances are you won't be seen till 6:45pm. So not only was that the stupidest business model for a company to have if they specialize in doing hair and makeup but the customer service was garbage. They were sooooooo rude about the entire situation. The receptionist didn't want to take any responsibility for the mix up, and the other guy was a complete prick in the manner that he spoke to us. To make matters worse they screwed up on the type of appointments, and then accused us of calling and canceling one of the makeup appointments!!!! I agree mistakes happen ALL the time, at first I wasn't upset until the prick came over to speak to us. The manner that they dealt with the entire situation was disgusting, unprofessional and ignorant. NOW I can't say anything about the makeup artist because well we didn't get anything done due to all the screw ups, however, my cousin got her hair done and well it was nothing impressive.",ANGRY,ANGRY
67935,"I went here alone when I first moved to Pittsburgh. Maybe it would've been a better experience had I not been alone, but I go out alone a lot, and I've never felt as awkward as I did when I walked into Gooski's. Everyone turned to look at me as I came in the door, and their eyes followed me until I took a seat in a booth by myself (do I have something on my face?). I grabbed a beer, sat and drank it and decided to leave rather than stay. It didn't feel very friendly to me.. but maybe I'll give it a second chance, based on some of these other reviews.",ANGRY,ANGRY
533445,"We went here